In [3]:
from z3 import *

NUM_ROWS = 1
NUM_TXS = 4


t_stats = [
    [(Int(f'k_{r_id}_{s_id}'), Int(f'v_{r_id}_{s_id}')) for s_id in range(0, NUM_TXS + 1)] for r_id in range(0, NUM_ROWS)
]

s = Solver()

s.add(And(t_stats[0][0][0] == 1, t_stats[0][0][1] == 0))
s.add(And(t_stats[0][NUM_TXS][0] == 1, t_stats[0][NUM_TXS][1] == 4))

transition_stats = [
    [((Int(f'k_{r_id}_{tx_id}_b'), Int(f'v_{r_id}_{tx_id}_b')), (Int(f'k_{r_id}_{tx_id}_a'), Int(f'v_{r_id}_{tx_id}_a')))
        for tx_id in range(1, NUM_TXS + 1)] 
        for r_id in range(0, NUM_ROWS)
]

for r_id in range(0, NUM_ROWS):
    # Tx1
    f_cond = Implies(transition_stats[r_id][0][0][1] == 0, And(transition_stats[r_id][0][1][1] == 1, transition_stats[r_id][0][1][0] == transition_stats[r_id][0][0][0]))
    f_not_cond = Implies(transition_stats[r_id][0][0][1] != 0, And(transition_stats[r_id][0][1][1] == transition_stats[r_id][0][0][1], transition_stats[r_id][0][1][0] == transition_stats[r_id][0][0][0]))
    s.add(And(f_cond, f_not_cond))
    
    # Tx2
    f_cond = Implies(transition_stats[r_id][1][0][1] == 1, And(transition_stats[r_id][1][1][1] == 2, transition_stats[r_id][1][1][0] == transition_stats[r_id][1][0][0]))
    f_not_cond = Implies(transition_stats[r_id][1][0][1] != 1, And(transition_stats[r_id][1][1][1] == transition_stats[r_id][1][0][1], transition_stats[r_id][1][1][0] == transition_stats[r_id][1][0][0]))
    s.add(And(f_cond, f_not_cond))

    # Tx3
    f_cond = Implies(transition_stats[r_id][2][0][1] == 2, And(transition_stats[r_id][2][1][1] == 3, transition_stats[r_id][2][1][0] == transition_stats[r_id][2][0][0]))
    f_not_cond = Implies(transition_stats[r_id][2][0][1] != 2, And(transition_stats[r_id][2][1][1] == transition_stats[r_id][2][0][1], transition_stats[r_id][2][1][0] == transition_stats[r_id][2][0][0]))
    s.add(And(f_cond, f_not_cond))

    # Tx4
    f_cond = Implies(transition_stats[r_id][3][0][1] == 3, And(transition_stats[r_id][3][1][1] == 4, transition_stats[r_id][3][1][0] == transition_stats[r_id][3][0][0]))
    f_not_cond = Implies(transition_stats[r_id][3][0][1] != 3, And(transition_stats[r_id][3][1][1] == transition_stats[r_id][3][0][1], transition_stats[r_id][3][1][0] == transition_stats[r_id][3][0][0]))
    s.add(And(f_cond, f_not_cond))

def gen_formula_link_states(s_id, tx_id, ba_idx):
    ret_list = []
    for r_id in range(NUM_ROWS):
        ret_list.append(t_stats[r_id][s_id][0] == transition_stats[r_id][tx_id - 1][ba_idx][0])
        ret_list.append(t_stats[r_id][s_id][1] == transition_stats[r_id][tx_id - 1][ba_idx][1])
    return And(ret_list)

for tx_id in range(1, NUM_TXS + 1):
    for s_id in range(0, NUM_TXS):
        s.add(And(Implies(gen_formula_link_states(s_id, tx_id, 0), gen_formula_link_states(s_id + 1, tx_id, 1))), 
              Implies(gen_formula_link_states(s_id + 1, tx_id, 1), gen_formula_link_states(s_id, tx_id, 0)))
        

f = Function('f', IntSort(), IntSort())

for i in range(1, 1 + NUM_TXS):
    for j in range(1, 1 + NUM_TXS):
        s.add(Implies(f(i) == j, gen_formula_link_states(j, i, 1)))

for i in range(1, 1 + NUM_TXS):
    s.add(And(f(i) >= 1, f(i) <= NUM_TXS))
    for j in range(1, i):
        s.add(f(i) != f(j))


print(s.check())


m = s.model()
print(m)

sat
[v_0_3_b = 2,
 v_0_2_a = 2,
 v_0_2 = 2,
 k_0_2_b = 1,
 k_0_4_b = 1,
 v_0_1_a = 1,
 v_0_2_b = 1,
 v_0_3_a = 3,
 v_0_1 = 1,
 k_0_2_a = 1,
 k_0_3 = 1,
 k_0_3_b = 1,
 k_0_3_a = 1,
 k_0_1_a = 1,
 k_0_2 = 1,
 v_0_4_b = 3,
 k_0_4_a = 1,
 k_0_1 = 1,
 v_0_1_b = 0,
 k_0_1_b = 1,
 v_0_4_a = 4,
 v_0_3 = 3,
 v_0_0 = 0,
 v_0_4 = 4,
 k_0_0 = 1,
 k_0_4 = 1,
 f = [2 -> 2, 3 -> 3, 4 -> 4, else -> 1]]
